Antes de empezar debemos importar los paquetes y módulos que vamos a necesitar durante este análisis. En concreto, vamos a importar el paquete de visualización *matplotlib*, el módulo *Pandas* para trabajar con datos tabulares y algunas funciones de *Spark*.

In [ ]:
%matplotlib inline

import pandas as pd

from matplotlib import pyplot as plt
from pyspark.sql import HiveContext
from pyspark.sql import functions as f

En las celdas siguientes configuramos un enlace para acceder a los datos de Hive y definimos dos Spark Dataframes que harán referencia a los datos de las dos tablas externas de Hive (voto_csv y usuario_csv).

In [ ]:
hive = HiveContext(sc)

sdf_votos = sqlContext.table('voto_csv')
sdf_usuarios = sqlContext.table('usuario_csv')

Una vez inicializados estos objetos en Spark, podemos mostrar su modelo de datos para comprobar que todo es correcto.

In [ ]:
sdf_votos

In [ ]:
sdf_usuarios

In [ ]:
sdf_usuarios.count()

# Estudio y limpieza de los datos de usuario

Para empezar, vamos a mostrar algunos registros de usuarios usando el método *show*

In [ ]:
sdf_usuarios.show(5)

Y podemos contar cuantos usuarios han participado en total usando la método *count()*.

In [ ]:
sdf_usuarios.count()

## País

Para empezar, vamos a generar un histograma con los paises de todos los usuarios que han participado usando las funcionalidades presentes en Spark. Para ello, ejecuta la celda siguiente:

In [ ]:
hist_usuarios_pais = sdf_usuarios.groupby('pais').count().toPandas()
hist_usuarios_pais.head()

La celda anterior agrupa los usuarios según su país y luego cuenta el número de usuarios que hay en cada grupo (país).
Podemos visualizar esta información con las funcionalidades de Matplotlib que están integradas en Pandas. Ejecuta la celda siguiente para mostrar el histograma como un gráfico de barras:

In [ ]:
hist_usuarios_pais.plot(kind='bar')

En el gráfico anterior, en el eje X nos muestra la posición en lugar del nombre del país. Para cambiar esto, nos conviene definir el índice del Pandas DataFrame usando el método *set_index*. Ejecuta la siguente celda para ver el resultado:

In [ ]:
hist_usuarios_pais.set_index('pais').plot(kind='bar')

Ahora podemos ver claramente que el país que más participantes ha tenido (más de 4000) ha sido México, y el que menos, parece ser Uruguay. Como no queda muy claro, vamos a ordenar los valores de este histograma usando el método *sort_values*. Ejecuta la celda siguiente para ver el resultado:

In [ ]:
hist_usuarios_pais.set_index('pais').sort_values('count').plot(kind='bar')

Ahora sí podemos ver claramente que Uruguay es el país que menos participantes ha tenido, aunque seguido bien de cerca por Panamá o Puerto Rico, entre otros.

## Edad

En segundo lugar, vamos a estudiar la distribución de edades de los participantes. Para ello, vamos a construir un histogramamuy similar al anterior pero, en este caso, en lugar de agrupar por país, vamos a agrupar por edad. Completa la linea siguiente para generar los datos del histograma:

In [ ]:
hist_usuarios_edad = sdf_usuarios.groupby('    ').count().toPandas()
hist_usuarios_edad.head()

Para mostrar los datos de este histograma usaremos otro tipo de gráfico, llamado "scatter". Si todo ha ido bien en la celda anterior, ejecuta la celda siguiente para visualizar el gráfico:

In [ ]:
hist_usuarios_edad.plot('edad', 'count', kind='scatter')

En este gráfico podemos ver como hay participantes menores de 10 años y mayores de 100. Más adelante nos va a interesar filtrar los votos de estos usuarios, pues pueden ser poco fiables. Vamos a ver cuantos usuarios hay que tengan menos de 10 años, para ello, ejecuta la celda siguiente:

In [ ]:
sdf_usuarios.filter('edad < 20').count()

Deberías haber obtenido un resultado de 8 usuarios menores de 10 años. Para saber cuántos usuarios mayores de 100 años han participado, completa la celda siguiente y ejecútala:

In [ ]:
sdf_usuarios.filter('edad >    ').count()

Deberías haber obtenido 7 como resultado.

Finalmente, vamos a generar el conjunto de datos limpio filtrando sólo los usuarios que tengan edades comprendidas entre los 10 y los 100 años, ambos incluidos. En la celda siguiente, generamos este conjunto de datos limpio y contamos cuantos usuarios nos han quedado.

In [ ]:
sdf_usuarios_limpio = sdf_usuarios.filter('edad >= 10 AND edad <= 100')
sdf_usuarios_limpio.count()

Deberían quedarnos $14919 - 8 - 7 = 14904$

# Estudio y limpieza de los datos de votos

Una vez analizados los datos de los usuarios, vamos a hacer lo mismo con los votos. Ejecuta la celda siguiente para ver información de algunos votos:

In [ ]:
sdf_votos.show(5)

Y podemos contar cuantos votos hemos recodigo en total usando la método *count()*.

In [ ]:
sdf_votos.count()

## Tiempo de respuesta

Ahora vamos a estudiar la distribución del tiempo de respuesta, para ver cómo de rápido han sido los participantes a la hora de clasificar las galaxias según su forma.

Como puedes observar en el resultado mostrado en la celda anterior, en este caso tenemos una complejidad adicional que es el hecho de que el tiempo de respuesta no es un valor con el cual podamos agrupar directamente, ya que no es un valor categórico y cada voto tiene un tiempo de respuesta distinto.

Para poder agrupar los distintos votos, deberemos crear categorías artificiales, por ejemplo agrupando los tiempos de respuesta en bloques de un segundo. Para llevar a cabo esta agrupación podemos usar la función *floor* en la que, dado un tiempo de respuesta, nos va a devolver la parte entera despreciando la parte decimal. De esta manera, varios votos que hayan tardado 3.425, 3.019 o 3.978 segundos van a quedar agrupados en la misma categoria, los que han tardado de 3 a 4 segundos.

Vamos a generar esta columna extra con el método *withColumn* aplicando esta función *floor*. Ejecuta la celda siguiente para ver esta nueva columna:

In [ ]:
sdf_votos.withColumn('segundos', f.floor('tiempo')).show(10)

Fíjate como a varios valores de la columna tiempo se les asigna el mismo número de segundos. Ahora ya sí podemos agrupar por esta columna nueva y generar el histograma.

In [ ]:
hist_votos_tiempo = sdf_votos.withColumn('segundos', f.floor('tiempo')).groupby('segundos').count().toPandas()
hist_votos_tiempo.head()

Visualiza ahora tu mismo este mismo histograma usando un gráfico de tipo "scatter". Sólo tienes que completar la celda inferior y ejecutarla.

In [ ]:
hist_votos_tiempo.plot('   ', '   ', kind = '   ')

Lo primero que podemos ver en este gráfico es que parece ser que hay clasificaciones donde el usuario se ha tomado su tiempo para decidirse. Hay bastantes votos en los que el usuario ha necesitado más de 60 segundos para decidirse, incluso algunos han tardado más de 300 segundos (5 minutos). Es muy probable que no nos interese tener en cuenta estos votos, ya que la información que contienen podría ser problemática.

Calcula el número de votos que han tomado 60 segundos o más en generarse completando y ejecutando la celda siguiente:

In [ ]:
sdf_votos.filter('tiempo >=    ').count()

Debería obtener el valor 2866.

Por otro lado, en la visualización anterior perdemos el detalle de lo que sucede para tiempos reducidos. Hay un parámetro *xlim* que se le puede pasar al método *plot* para especificar el rango que queremos que nos muestre en el eje X. Añade este parámetro con el valor [-1, 30] en la siguiente celda y vuelve a generar el gráfico.

In [ ]:
hist_votos_tiempo.plot('segundos', 'count', kind = 'scatter', xlim=[-1,30])

Si lo has hecho correctamente podrás observar como hay algunas clasificaciones que se han realizado en menos de un segundo, ya que aparecen agrupadas en la coordenada 0. Calcula el número de votos que se han realizado en menos de un segundo completando y ejecutando la celda siguiente:

In [ ]:
sdf_votos.filter('tiempo <    ').count()

Deberías obtener el valor 2639.

Estos votos también podría interesarnos no tenerlos en cuenta, ya que es bastante probable que el usuario no le haya prestado suficiente atención.

Para ir finalizando, vamos a generar un conjunto de datos limpio que sólo contenga los votos en los que el tiempo de respuesta sea mayor de un segundo y menor de sesenta. Completa la celda inferior para generar este conjunto de datos limpio y mostrar algunos registros.

In [ ]:
sdf_votos_limpio = sdf_votos.filter('tiempo >=     AND tiempo <=    ')
sdf_votos_limpio.show(5)

Y, por último, usa el método *count* en este conjunto de datos limpio para calcular cuantos votos nos han quedado después de sanear los datos.

In [ ]:
sdf_votos_limpio.count()

# Combinación de los datos limpios

Ahora que tenemos los dos conjuntos de datos limpios, vamos a combinarlos para estudiarlos en su conjunto. Para ello usaremos el método *join* para combinar ambos por la columna que tienen en común, que es el identificador del usuario.

In [ ]:
sdf_combinado = sdf_votos_limpio.join(sdf_usuarios_limpio, 'id_usuario')
sdf_combinado.show(10)

Al combinar estos dos conjuntos de datos vamos a generar un conjunto de datos aún más limpio, ya que en la combinación se van a descartar todos los votos de aquellos usuarios que hemos filtrado por estar fuera del rango de edad.

Calcula el número de registros del conjunto de datos combinados con el método *count*.

In [ ]:
sdf_combinado.count()

Deberías obtener el resultado 373451, lo que nos indica que en la combinación de los dos conjuntos de datos, se han descartado $373644 - 373451 = 193$ votos adicionales.

## Votos según el país

Ahora que tenemos los datos combinados, podemos sacar información adicional sobre estos datos. Por ejemplo, podemos calcular el histograma de número de votos por país. En la celda siguiente tienes el código para generar los datos para el histograma del número de votos por país.

In [ ]:
hist_votos_pais = sdf_combinado.groupby('pais').count().toPandas()
hist_votos_pais.head()

In [ ]:
hist_votos_pais.plot(kind='bar')

Al igual que nos pasó al mostrar los datos del número de usuarios por país, en este gráfico nos aparece la posición del país en lugar de su nombre. Completa la celda inferior usando los métodos *set_index* y *sort_values* para generar una visualización donde aparezca el nombre el país en el eje X y los valores del histograma estén ordenados.

In [ ]:
hist_votos_pais.set_index('    ').sort_values('     ').plot(kind='bar')

Como era de esperar México sigue liderando en el recuento de votos al tener también el mayor número de usuarios participantes.

El problema con esta visualización es que no podemos determinar si los usuarios de México o los de cualquier otro país han sido más productivos que el resto. Es decir, si los usuarios de algún país han sido generado, de media, más votos de los de otros países. Para saber esto, podemos calcular el número medio de votos por usuario de cada país, dividiendo los datos de los dos histogramas (el de número de votos por país y el de número de usuarios por país).

In [ ]:
hist_media_voto_pais = hist_votos_pais.set_index('pais') / hist_usuarios_pais.set_index('pais')
hist_media_voto_pais.head()

Vamos a visualizar esta información ahora con un gráfico de barras.

In [ ]:
hist_media_voto_pais.plot(kind='bar')

A la vista de la información presentada, no parece haber diferencias significativas en la participación de los usuarios de cada país. De media, cada participante ha clasificado la forma de unas 25 galaxias.

## Votos según la edad

Sucederá lo mismo si estudiamos el número de votos medio segun la franja de edad? Completa las celdas siguientes para averiguarlo.

In [ ]:
hist_votos_edad = sdf_combinado.groupby('    ').count().toPandas()
hist_votos_edad.head()

In [ ]:
hist_votos_edad.plot('    ', '     ', kind='scatter')

A primera vista vemos una distribución del número de votos según la edad muy similar a la distribución de usuarios según la edad. Esto era esperable ya que las franjas con más usuarios también serán las que deberían haber generado más clasificaciones. Para poder salir de dudas, es necesario hacer como en el caso anterior y dividir la información de los dos histogramas para calcular el número medio de votos segun la edad.

In [ ]:
hist_media_voto_edad = hist_votos_edad.set_index('edad') / hist_usuarios_edad.set_index('edad')
hist_media_voto_edad.head(10)

Si ves algun valor 'NaN' en la tabla anterior, no te preocupes, corresponden a datos de usuarios o votos que han sido filtrados (por edad o tiempo).

In [ ]:
hist_media_voto_edad.reset_index().plot('edad', 'count', kind='scatter')

Fíjate bien la gráfica anterior, pues presenta un comportamiento particular que deberás saber explicar más adelante.


# Generar la clasificación promedio

Una vez hemos limpiado y estudiado los datos y estamos conformes con su validez, es el momento de generar la clasificación ideal de cada una de las galaxias. Para ello, vamos a promediar el voto de todos los usuarios sobre cada una de las galaxias. Al habernos preocupado previamente de guardar la forma como un número (0 en caso de votar elíptica, y 1 en caso de votar espiral), esta operación va a ser muy sencilla. Si la mayoría de los usuarios han votado una imagen de una galaxia como elíptica, la media de sus votos debería ser cercana a 0. En caso contrario, si la mayoría de los votos han marcado una galaxia como espiral, la media de la columna forma debería ser cercana a 1.

Antes de continuar, recordemos el aspecto del conjunto de datos combinado con el método show.

In [ ]:
sdf_combinado.show(5)

Para generar la clasificación promedio, debemos agrupar por la columna que identifica únicamente a cada galaxia y calcular la media del campo forma. Completa la celda siguiente y ejecútala:

In [ ]:
sdf_clasificado = sdf_combinado.groupby('          ').agg(f.avg('forma').alias('forma'))
sdf_clasificado.show(5)

En la celda anterior podemos ver como para cada galaxia hemos obtenido su forma esperada segun la media de los votos. El último paso que nos queda es guardar esta información en otra tabla, que va a ser usada en la semana siguiente para entrenar un clasificador automatizado.

In [ ]:
sdf_clasificado.write.saveAsTable('target', mode='overwrite')